In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit


### Install adk

In [2]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00


In [3]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
MODEL_NAME = "gemini-2.0-flash"
#MODEL_NAME = "gemini-2.5-pro-preview-03-25"

BUCKET_URI = f"gs://agent-0417"

#  model = "gemini-2.5-pro-preview-03-25"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initial set up

In [ ]:
# @title Create a bucket.
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0417/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0417' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [ ]:
# @title Initialize Vertex AI with Staging Bucket.
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Agent on adk

In [ ]:
#@title Build a Agent with adk.
from google.adk.agents import Agent
from google.adk.tools import google_search

agent = Agent(
    name="search_assistant",
    model=MODEL_NAME,
    instruction="You are a helpful assistant. Answer user questions using Google Search when needed.",
    description="An assistant that can search the web.",
    tools=[google_search] # Use Google search tool
)


In [ ]:
from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(agent=agent)

for event in app.stream_query(
   user_id="shins",  # Required
   message="What is the exchange rate from US dollars to Korean currency today ?",
):
   print(event)
   print(event['content']['parts'][0]['text'])

{'content': {'parts': [{'text': "As of April 22, 2025, here are the approximate exchange rates for 1 US dollar (USD) to South Korean Won (KRW):\n\n*   **Trading Economics:** 1 USD = 1,423.85 KRW\n*   **XE.com:** 1 USD = 1,428.23 KRW\n*   **Revolut:** 1 USD = 1,425.09 KRW\n\nIt's worth noting that exchange rates are constantly changing, so it's best to check with a currency converter or your bank for the most up-to-date information.\n"}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'tradingeconomics.com', 'title': 'tradingeconomics.com', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAK1cB-vNidKrG4_71niC6BpU5K3VnTix6oPB0aZhmWL43Hm5IJW-JCNVH7jEKbkAHec5VdyUVdFPK2zLJmQgg8sZ56jqtDQG7F8awiBVS6IQEUbDgIyYKMyTRV12epdupLVCwlvZ-ckhnWXZf8='}}, {'web': {'domain': 'xe.com', 'title': 'xe.com', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAKUokHgY66Lta9rq9RoeMGhoooPCe3hwd9p0j9-9cJnA_HekUQoQnIOpEItbvdVt5T3c

## Use a Tool

In [ ]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "KRW",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain
    exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code).
            Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code).
            Defaults to "KRW" (KRW).
        currency_date: The date for which to retrieve the exchange rate.
            Defaults to "latest" for the most recent exchange rate data.
            Can be specified in YYYY-MM-DD format for historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
            Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24",
                "rates": {"EUR": 0.95534}}
    """
    import requests
    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [ ]:
get_exchange_rate(currency_from="USD", currency_to="KRW")


{'amount': 1.0, 'base': 'USD', 'date': '2025-04-22', 'rates': {'KRW': 1425.18}}

In [ ]:
from google.adk.agents import Agent

agent = Agent(
    model=MODEL_NAME,
    name='currency_exchange_agent',
    tools=[get_exchange_rate],
)

In [ ]:
from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(agent=agent)

for event in app.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won on 2025-04-17?",
):
   print(event)


{'content': {'parts': [{'function_call': {'id': 'adk-b790653a-ec7a-4057-8c65-0e6209713263', 'args': {'currency_to': 'KRW', 'currency_from': 'USD', 'currency_date': '2025-04-17'}, 'name': 'get_exchange_rate'}}], 'role': 'model'}, 'invocation_id': 'e-547d80f9-c494-41ab-8caa-08b8b058ffbb', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'Sg4rDF51', 'timestamp': 1745369095.905716}
{'content': {'parts': [{'function_response': {'id': 'adk-b790653a-ec7a-4057-8c65-0e6209713263', 'name': 'get_exchange_rate', 'response': {'amount': 1.0, 'base': 'USD', 'date': '2025-04-17', 'rates': {'KRW': 1416.48}}}}], 'role': 'user'}, 'invocation_id': 'e-547d80f9-c494-41ab-8caa-08b8b058ffbb', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'IrDc85SI', 'timestamp': 1745369097.32742}
{'content': {'parts': [{'text': 'The excha

In [ ]:
for event in app.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won currency on 2025-04-15?",
):
    print(event)

## Deploy ADK on Agent Engine

In [ ]:
# @title Deploy your agent on Vertex AI

from vertexai import agent_engines

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_create

remote_agent = agent_engines.create(
    app,
    display_name="currency agent with adk for AgentSpace",
    gcs_dir_name = "ai-agent-adk",
    description="This is a simple agent with adk on agent engine.",
    requirements=[
        "google-adk",
        "cloudpickle==3.0",
    ],
    extra_packages = []
)

INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.88.0', 'cloudpickle': '3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-adk', 'cloudpickle==3.0']
INFO:vertexai.agent_engines:Using bucket agent-0417
INFO:vertexai.agent_engines:Wrote to gs://agent-0417/ai-agent-adk/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://agent-0417/ai-agent-adk/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://agent-0417/ai-agent-adk/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/5442468208281911296/operations/2909372191604736000
INFO:vertexai.agent_engines:View progress and logs at https://console.cloud.google.com/logs/query?project=ai-hangsik
INFO:vertexai.agent_engines:AgentEngine created. Resource

In [ ]:
# @title Query from remote engine.

for event in remote_agent.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won currency on 2025-04-15?",
):
    print(event)

{'content': {'parts': [{'text': 'Based on the search results for the USD to KRW exchange rate on April 15, 2025, here are the findings:\n\n*   One source indicates that on **Tuesday, April 15, 2025, 1 US Dollar (USD) was equal to 1428 South Korean Won (KRW)**. It also mentions a range with 1421 and 1429 KRW for that day.\n*   Another source (Investing.com) lists the closing price for April 15, 2025, as **1 USD = 1,413.71 KRW**. It also provides the opening rate (1,427.94 KRW), the high (1,432.74 KRW), and the low (1,412.39 KRW) for that day.\n*   A third source (XE.com) shows a rate of **1 USD = 1,428.11 KRW** around 22:02 UTC on April 15, 2025, and another rate of **1 USD = 1,427.95 KRW** at 23:25 UTC on the same day.\n*   A fourth source (Foreign Exchange UK) lists **1 USD = 1427.9909 KRW** for April 15, 2025.\n*   OFX provides a historical average rate of **1 USD = 1452.0046 KRW** for April 15, 2025.\n*   YCharts indicates a rate of **0.0007 USD to 1 KRW** for April 15, 2025, which 

## Register Agent Engine to AgentSpace.

In [4]:
%%bash
export AGENT_DISPLAY_NAME="Currency agent"
export AGENT_DESCRIPTION="The agent can change currency"
export AGENT_ID="corp_currency_agent"
export AS_APP="agentspace-0423_1745357547756"

export REASONING_ENGINE="projects/721521243942/locations/us-central1/reasoningEngines/5442468208281911296/operations/2909372191604736000"
export PROJECT_NUMBER="721521243942"
export PROJECT_ID="ai-hangsik"

# echo "REASONING_ENGINE: $REASONING_ENGINE"
# echo "PROJECT_NUMBER: $PROJECT_NUMBER"
# echo "PROJECT_ID: $PROJECT_ID"

curl -X PATCH -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "x-goog-user-project: ${PROJECT_ID}" \
https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant?updateMask=agent_configs -d '{
    "name": "projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant",
    "displayName": "Default Assistant",
    "agentConfigs": [{
      "displayName": "'"${AGENT_DISPLAY_NAME}"'",
      "vertexAiSdkAgentConnectionInfo": {
        "reasoningEngine": "'"${REASONING_ENGINE}"'"
      },
      "toolDescription": "'"${AGENT_DESCRIPTION}"'",
      "icon": {
        "uri": "https://fonts.gstatic.com/s/i/short-term/release/googlesymbols/corporate_fare/default/24px.svg"
      },
      "id": "'"${AGENT_ID}"'"
    }]
  }'


{
  "name": "projects/721521243942/locations/global/collections/default_collection/engines/agentspace-0423_1745357547756/assistants/default_assistant",
  "displayName": "Default Assistant",
  "webGroundingType": "WEB_GROUNDING_TYPE_GOOGLE_SEARCH",
  "googleSearchGroundingEnabled": true,
  "vertexAiAgentConfigs": [
    {
      "displayName": "Travel Agent",
      "name": "projects/ai-hangsik/locations/us-central1/agents/ddf6c49a-7ece-442a-bcff-c3e33cefe0e9",
      "toolDescription": "response to the answer"
    }
  ],
  "customerPolicy": {},
  "vertexAiSearchToolConfig": {}
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1254    0   582  100   672   1294   1494 --:--:-- --:--:-- --:--:--  2786


## End of notebook